#### *Instalando as bibliotecas que serão necessárias*

In [ ]:
# Instala a biblioteca "pandas" que será usada para manipulação de dados.
!pip install pandas

In [ ]:
# Instala a biblioteca "numpy" que será usada para operações numéricas.
!pip install numpy

In [ ]:
# Instala a biblioteca "yfinance" que será usada para obter dados financeiros.
!pip install yfinance

In [ ]:
# Instala a biblioteca "plotly" que será usada para a exibição de gráficos. 
!pip install plotly

#### *Importando as bibliotecas que serão necessárias*

In [ ]:
# Importa a biblioteca "pandas", que será usada para manipulação de dados.
import pandas as pd

# Importa a biblioteca "numpy", que será usada para operações numéricas.
import numpy as np

# Importa a biblioteca "yfinance" para obter alguns dados de certas ações.
import yfinance as yf

# Importa o módulo "graph_objects" da biblioteca "plotly" para lidar com a exibição de gráficos.
import plotly.graph_objects as go

# Importa o módulo datetime da biblioteca "datetime" para lidar com datas.
from datetime import datetime, timedelta

# Importa a biblioteca "math" para lidar com certas operações matemáticas.
import math

# Importa o módulo Optional da biblioteca "typing" para lidar com a padronização dos tipos de parâmetros opcionais das funções.
from typing import Optional

# Importa a classe Ticker da "biblioteca" Ticker para lidar com ativos financeiros. 
from Ticker import Ticker

# Importa a classe Model da "biblioteca" Model para lidar com redes neurais do tipo LSTM.
from Model import Model

# Importa o módulo MinMaxScaler da biblioteca "sklearn", que será utilizado para realizar a "desnormalização" 
# das variáveis target que foram usadas no modelo LSTM.
from sklearn.preprocessing import MinMaxScaler

#
from scipy import stats


#### *Definindo funções para processar os arquivos .xlsx*

In [ ]:
def process_file_and_get_results(filename: str) -> pd.DataFrame:
    '''
        Description:

        Args:

        Return:
    '''

    # Ler todas as planilhas do arquivo Excel
    dfs = pd.read_excel(filename, sheet_name=None)
    
    #
    first_trading_day = dfs['Período 1'][dfs['Período 1'].columns[0]][0].split(" - ")[0]
    #
    first_trading_day = np.datetime64(first_trading_day)
    
    #
    results = pd.DataFrame({
        "Retorno Long": 0,
        "Retorno Short": 0,
        "Retorno Total": 0
    }, index=[first_trading_day])
    
    # Acessar e trabalhar com cada DataFrame
    for period, df in dfs.items():
        if not df.empty:
            #
            period_last_trading_day = df[df.columns[0]][0].split(" - ")[1]
            #
            period_last_trading_day = np.datetime64(period_last_trading_day)
            #
            tickers_to_trade = math.ceil(df.shape[0]*0.1)
            #
            period_long_return = df.iloc[:tickers_to_trade]['Retorno Real'].mean() # Trocar por um .sum() ?
            #
            period_short_return = df.iloc[df.shape[0] - tickers_to_trade:]['Retorno Real'].mean()
            #
            result = pd.DataFrame({
                "Retorno Long": period_long_return,
                "Retorno Short": period_short_return*(-1),
                "Retorno Total": period_long_return + period_short_return*(-1)
            },index=[period_last_trading_day])
            #
            results = pd.concat([results, result])
    
    return results
            

In [ ]:
def process_files_and_get_results(filesname: list) -> list:
    '''
        Description:

        Args:

        Return:
    '''
    
    #
    results = []
    
    #
    for xlsx_number, xlsx in enumerate(filesname):
        #
        result = process_file_and_get_results(xlsx)
        #
        result.name = f"Retornos da Estratégia na Previsão {xlsx_number}"
        #
        results.append(result)
        
    return results

In [ ]:
def get_average_result(results: list) -> pd.Series:
    '''
        Description:

        Args:

        Return:

    '''
    
    # Garantir que todos os resultados em results tenham o mesmo tamanho (para poder inicializar o average_result).
    
    
    average_result = np.zeros(len(results[0]))
        
    for result in results:
        average_result += result
    
    average_result = average_result/len(results)
    
    average_result.name = "Retornos da Estratégia"
    
    return average_result

In [ ]:
def get_ticker_standardized_returns(symbol, reference_data, strategy_time_period: str, 
                                    standardized_returns_name="", standardized_cumulative_returns_name="") -> pd.Series:
    '''
        Description:

        Args:

        Return:
    '''

    #
    period = 5 if (strategy_time_period == "W") else 20
    #
    periods_number = len(reference_data)
    
    #
    result = np.zeros(periods_number)
    
    #
    data_extraction_initial_day = reference_data.index[0]
    #
    data_extraction_final_day = reference_data.index[-1] + timedelta(1) # O time delta 1 é por que o yf não inclui o último dia de 
                                                                        # negociação passado como parâmetro. Logo, somando um timedelta(1),
                                                                        # tal dia será incluido na extração de dados do S&P500 index.
                                                                            
    #
    adj_close = yf.download(symbol, data_extraction_initial_day, data_extraction_final_day)['Adj Close']
    
    #
    result[0] = np.log((adj_close[period - 1])/adj_close[0])*100
    #
    for period_number in range(1, periods_number):
        #
        result[period_number] = np.log((adj_close[(period)*period_number - 1 + (period)])/(adj_close[(period)*period_number]))*100
        
    #
    result = pd.Series(result, reference_data.index)
    #
    cumulative_result = result.cumsum()
    #
    result.name = standardized_returns_name
    #
    cumulative_result.name = standardized_cumulative_returns_name
    
    return result, cumulative_result

#### *Definindo algumas variáveis que serão importantes para a exibição de resultados*

In [ ]:
setup = {
    #
    "monthly_results_sheets": ['resultados_portfolio_mensal_full_1.xlsx', 'resultados_portfolio_mensal_full_2.xlsx',
                               'resultados_portfolio_mensal_full_3.xlsx', 'resultados_portfolio_mensal_full_4.xlsx',
                               'resultados_portfolio_mensal_full_5.xlsx'],
    #
    "weekly_results_sheets": ['resultados_portfolio_semanal_full_1.xlsx']
}

In [ ]:
#
monthly_all_results = process_files_and_get_results(setup['monthly_results_sheets'])

In [ ]:
monthly_all_results

In [ ]:
monthly_results = [element['Retorno Total'] for element in monthly_all_results]

for index, result  in enumerate(monthly_results): 
    result.name = f"Retornos da Estratégia na Previsão {index}"

In [ ]:
monthly_results

In [ ]:
#
weekly_results = process_file_and_get_results(setup['weekly_results_sheets'][0]) # O .xlsx semanal é processado desse jeito pois, por hora,
                                                                                 # só possuímos 1 dele.

In [ ]:
#
average_monthly_results = get_average_result(monthly_results)

In [ ]:
average_monthly_results.sum()

In [ ]:
# Posso usar qualquer "result" em "results" como reference_data, já que, ambos "result" possuem as mesmas datas como índices.

sp500_index_period_returns, sp500_index_cumulative_returns = get_ticker_standardized_returns('^GSPC', monthly_results[0], "M",
                                                                                             "Retornos do S&P500 Index",
                                                                                             "Retorno Acumulado do S&P500 Index")

In [ ]:

# Exemplo de como exportar para um arquivo Excel com duas planilhas
with pd.ExcelWriter('sp500_returns.xlsx') as writer:
    sp500_index_period_returns.to_excel(writer, sheet_name='Period Returns')
    sp500_index_cumulative_returns.to_excel(writer, sheet_name='Cumulative Returns')


#### *Exibindo e explicando as variáveis definidas na seção anterior*

In [ ]:
#
monthly_results[0]

In [ ]:
#
monthly_results[0]

In [ ]:
#
average_monthly_results

In [ ]:
#
weekly_results

In [ ]:
#
monthly_results[0]

In [ ]:
#
sp500_index_period_returns

In [ ]:
#
sp500_index_cumulative_returns

#### *Definindo algumas funções que serão úteis para plotar gráficos*

In [ ]:
def plot_time_serie_line_graph(serie: pd.Series, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "", annotation_text: Optional[str] = "") -> None:
    '''
        Description:
            Essa função é responsável por realizar o plot de uma série temporal.
        Args:
            serie (pd.Series): Série temporal cujos dados serão utilizados para o plot. Os valores de tal série temporal estarão no eixo y,
            enquanto os índices dessa série estarão no eixo x.
            title (string):
            xaxis_title (string):
            yaxis_title (string):
            
        Return:
            Essa função plota a série temporal, mas não retorna nada.
    '''
    
    # Cria a figure onde a série temporal em questão será plotada.
    fig = go.Figure()
    
    # Adiciona o gráfico da série temporal em questão ao plot.
    fig.add_trace(go.Scatter(
        # Define os valores do eixo x do plot.
        x = serie.index,
        # Define os valores do eixo y do plot.
        y = serie.values,
        # Define o tipo do gráfico que será plotado.
        mode = "lines"
    ))
        
    # Adiciona a anotação ao gráfico
    fig.add_annotation(
        x=1, y=1, 
        text=annotation_text, 
        showarrow=False, 
        font=dict(size=12, color="black"), 
        align="center", 
        xref="paper", yref="paper", 
        bgcolor="white", opacity=0.7
    )
    
    # Adiciona algumas legendas ao plot.
    fig.update_layout(
        # Adiciona um título ao plot.
        title = serie_title,
        # Adiciona uma legenda ao eixo x do plot.
        xaxis_title = serie_xaxis_title,
        # Adiciona uma legenda ao eixo y do plot.
        yaxis_title = serie_yaxis_title
    )
    
    # Exibe o plot criado.
    fig.show()

In [ ]:
def plot_multiples_time_series_line_graphs(data_list: list, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "", annotation_text: Optional[str] = "") -> None:
    '''
        Description:
            Esta função exibe em um mesmo plot múltiplos gráficos de séries temporais. 
        Args:
            data_list (list): Lista contendo as séries temporais cujos gráficos serão exibidos no plot. Os valores de tais séries temporais
                              estarão no eixo y, enquanto os índices dessas séries estarão no eixo x.
            title (string) [Optional]: Título do plot.
            xaxis_title (string) [Optional]: Título do eixo x do plot.
            yaxis_title (string) [Optional]: Título do eixo y do plot.
        Return:
            None: A função exibe os gráficos, mas não retorna nenhum valor.  
        Errors:
            TypeError: É esperado que todas os elementos da lista "data_list" sejam objetos do tipo pd.Series, isto é, que sejam séries temporais.
            ValueError: É esperado que todas as séries temporais presentes na variável "data_list" possuam os mesmos índices. 
            TypeError: É esperado que todas as séries temporais presentes na variável data_list possuam um atributo "name".
    '''    

    # Verifica se todos os elementos presentes na lista "data_list" são séries temporais.
    are_all_data_time_series = all(isinstance(df, pd.Series) for df in data_list)
    
    # Retorna um erro caso algum dos dados presentes na variável "data_list" não seja uma série temporal
    if not are_all_data_time_series:
        raise TypeError("Todos os dados presentes no parâmetro 'data_list' devem ser séries temporais.")
    
    # Verifica se as séries temporais presentes na variável "data_list" possuem os mesmos índices.
    are_all_index_equal = all(df.index.equals(data_list[0].index) for df in data_list)
    
    # Retorna um erro caso as séries temporais possuam índices diferentes.
    if not are_all_index_equal:
        raise ValueError("Todos as séries temporais devem possuir os mesmos índices.")
    
    # Verifica se todas as séries temporais presentes na variável "data_list" possuem o atributo "name".
    all_time_series_have_names = all(hasattr(df,"name") for df in data_list)
    
    # Retorna um erro caso uma das séries temporais presentes na variável "data_list" não possua o atributo "name".
    if not all_time_series_have_names:
        raise TypeError("Todas as séries temporais devem possuir o atributo 'name'")
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data_list[0].index.tolist() # Observe que só podemos fazer isso pois temos certeza que todas as séries temporais possuem os mesmos índices.
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for time_serie in data_list:
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=time_serie.values, mode="lines", name=time_serie.name))
    
    # Adiciona a anotação ao gráfico
    fig.add_annotation(
        x=1, y=1, 
        text=annotation_text, 
        showarrow=False, 
        font=dict(size=12, color="black"), 
        align="center", 
        xref="paper", yref="paper", 
        bgcolor="white", opacity=0.7
    )
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        # Seta um título para o plot.
        title=serie_title,
        # Seta um título para o eixo x do plot.
        xaxis_title=serie_xaxis_title,
        # Seta um título para o eixo y do plot.
        yaxis_title=serie_yaxis_title,   
        #
        legend=dict(
            x=0.005,  # Posição horizontal dentro do gráfico (0 = esquerda, 1 = direita)
            y=0.995,  # Posição vertical dentro do gráfico (0 = inferior, 1 = superior)
            traceorder='normal',  # Ordem de exibição das linhas
        )
    )

    # Exibe o gráfico criado.
    fig.show()

#### ***Exibindo os Resultados***

##### **Retornos Mensais da Estratégia Vs. Retornos Mensais do S&P500 Index**

In [ ]:
plot_multiples_time_series_line_graphs([average_monthly_results, sp500_index_period_returns], "Retornos Mensais da Estratégia Vs. Retornos Mensais do S&P500 Index", "Data", "Retornos Percentuais   (Log.)")

##### **Retorno Acumulado da Estratégia vs. Retorno Acumulado do S&P500 Index**

In [ ]:
#
average_monthly_cumulative_returns = average_monthly_results.cumsum()
#
average_monthly_cumulative_returns.name = "Retorno Acumulado da Estratégia"
#
plot_multiples_time_series_line_graphs([average_monthly_cumulative_returns, sp500_index_cumulative_returns], "Retorno Acumulado da Estratégia vs. Retorno Acumulado do S&P500 Index", "Data", "Retorno Acumulado    (Log.)")

##### **Relação entre os Retornos da Estratégia e os Retornos do S&P500 Index**

In [ ]:
# 
fig = go.Figure(data=go.Scatter(
    #
    x=sp500_index_period_returns,
    #
    y=average_monthly_results,
    #
    mode='markers',
    name="Pontos de Dispersão"
))

# Calcula a linha de tendência (regressão linear)
slope, intercept = np.polyfit(sp500_index_period_returns, average_monthly_results, 1)  # Retorna o coeficiente angular (slope) e o intercept
line = slope * sp500_index_period_returns + intercept  # Equação da linha de tendência

# Adiciona a linha de tendência
fig.add_trace(go.Scatter(
    x= sp500_index_period_returns, 
    y=line, 
    mode='lines', 
    name='Linha de Tendência', 
    line=dict(color='red', width=2, dash='dash')
))

# Adiciona títulos e rótulos
fig.update_layout(
    title="Relação entre os Retornos Mensais da Estratégia e os Retornos Mensais do S&P500 Index",
    xaxis_title="Retornos Percentuais do S&P500 Index    (Log.)",
    yaxis_title="Retornos Percentuais da  Estratégia    (Log.)",
    #
    legend=dict(
        x=0.005,  # Posição horizontal dentro do gráfico (0 = esquerda, 1 = direita)
        y=0.995,  # Posição vertical dentro do gráfico (0 = inferior, 1 = superior)
        traceorder='normal',  # Ordem de exibição das linhas
    )
)

# Exibe o gráfico
fig.show()

##### **Gráfico de barras dos Retornos da Estratégia vs. Retornos do S&P500 Index**

In [ ]:
# Dados de exemplo
x_values = average_monthly_results.index
y_values_1 = average_monthly_results
y_values_2 = sp500_index_period_returns

#
strategy_cumulated_value = np.exp((average_monthly_results/100).cumsum())
strategy_cumulated_peak = strategy_cumulated_value.cummax()
strategy_max_drawdown = ((strategy_cumulated_value - strategy_cumulated_peak)/strategy_cumulated_peak).min()*100

#
sp500_index_cumulated_value = np.exp((sp500_index_period_returns/100).cumsum())
sp500_index_cumulated_peak = sp500_index_cumulated_value.cummax()
sp500_index_max_drawdown = ((sp500_index_cumulated_value - sp500_index_cumulated_peak)/sp500_index_cumulated_peak).min()*100

# Retornos médios da estratégia e do S&P 500
strategy_mean = y_values_1.mean()
sp500_mean = y_values_2.mean()

# Definindo as cores das barras
color_strategy = "#006DAA"  # Cor das barras para a estratégia
color_sp500 = "#E4572E"  # Cor das barras para o S&P 500

# Criação das barras empilhadas
fig = go.Figure()

# Primeira barra (base) com cor definida
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_1,
    name=average_monthly_results.name,
    marker_color=color_strategy  # Definindo a cor da barra para a estratégia
))

# Segunda barra (sobreposta na anterior) com cor definida
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_2,
    name=sp500_index_period_returns.name,
    marker_color=color_sp500  # Definindo a cor da barra para o S&P 500
))

# Adiciona a anotação abaixo da legenda com a cor de fundo igual à cor das barras
fig.add_annotation(
    x=1.205, y=0.35,  # Ajuste y para colocar abaixo da legenda
    text=(
        f"Média: {strategy_mean:.2f}%<br>"
        f" Desvio padrão: {y_values_1.std():.2f}% <br>"
        f" Drawdown máximo {strategy_max_drawdown:.2f}% "
    ),
    showarrow=False,
    font=dict(size=12, color="white"),
    xref="paper", yref="paper",  # Define a posição em coordenadas relativas
    bgcolor=color_strategy,  # Usando a cor das barras da estratégia
    opacity=1,
    align="center"
)

# Adiciona a anotação para o S&P 500 abaixo da legenda
fig.add_annotation(
    x=1.215, y=0.7,  # Ajuste y para colocar abaixo da legenda
    text=(
        f"Média: {sp500_mean:.2f}%<br>"
        f" Desvio padrão: {y_values_2.std():.2f}% <br>"
        f" Drawdown máximo {sp500_index_max_drawdown:.2f}% "
    ),
    showarrow=False,
    font=dict(size=12, color="white"),
    xref="paper", yref="paper",  # Define a posição em coordenadas relativas
    bgcolor=color_sp500,  # Usando a cor das barras do S&P 500
    opacity=1,
    align="center"
)

# Configuração do layout para barras empilhadas
fig.update_layout(
    barmode='stack',
    title='Retornos Mensais da Estratégia vs. Retornos Mensais do S&P500 Index',
    xaxis_title='Data',
    yaxis_title='Retornos Percentuais    (Log.)'
)

# Exibe o gráfico
fig.show()


##### **Histograma dos Retornos da Estratégia com Curva Normal Ajustada**

In [ ]:
# Suponha que 'average_monthly_results' seja o seu vetor de retornos da estratégia
returns = average_monthly_results  # Substitua com os seus dados de retornos

# Ajusta a distribuição normal aos dados (média e desvio padrão)
mean, std_dev = stats.norm.fit(returns)

# Cria o histograma
hist_data = go.Histogram(
    x=returns,
    nbinsx=30,  # Defina o número de bins conforme necessário
    name='Histograma dos Retornos',
    opacity=0.75,
    histnorm='probability'  # Normaliza o histograma para a probabilidade
)

# Cria a curva normal ajustada
x_range = np.linspace(min(returns), max(returns), 1000)
y_range = stats.norm.pdf(x_range, mean, std_dev)  # Calcula a função de densidade da normal

# Adiciona a curva normal ao gráfico
normal_curve = go.Scatter(
    x=x_range,
    y=y_range,
    mode='lines',
    name='Curva Normal Ajustada',
    line=dict(color='red', width=2)
)

# Calcular a assimetria e a curtose
skewness = stats.skew(returns)
kurtosis = stats.kurtosis(returns)

# Cria a figura com histograma e curva normal
fig = go.Figure(data=[hist_data, normal_curve])

# Adicionar anotações de assimetria e curtose
fig.add_annotation(
    x=0.995, y=0.995, 
    text=f'     Assimetria: {skewness:.2f}<br>Curtose: {kurtosis:.2f}', 
    showarrow=False, 
    font=dict(size=12, color="black"), 
    align="center", 
    xref="paper", yref="paper", 
    bgcolor="white", opacity=0.8
)

# Atualiza o layout com títulos e rótulos
fig.update_layout(
    title="Histograma dos Retornos Mensais da Estratégia com Curva Normal Ajustada",
    xaxis_title="Retornos Percentuais    (Log.)",
    yaxis_title="Frequência (Probabilidade)",
    bargap=0.1,  # Ajusta o espaçamento entre as barras
    #
    showlegend=False
)

# Exibe o gráfico
fig.show()


*Assimetria -> quanto mais próximo de 0, mais "normal" é a distribuição*.

*Curtose -> quanto mais próximo de 3, mais "normal" é a distribuição*.

A assimetria de 0,37 indica que a distribuição dos retornos é ligeiramente assimétrica para a direita (cauda direita um pouco mais longa ou mais "gorda"). Isso significa que, em média, existem algumas observações com retornos maiores, mas não de forma extrema. A assimetria é positiva, mas o valor de 0,37 é relativamente baixo, indicando que a distribuição não é fortemente inclinada, mas apresenta um pequeno desvio da simetria perfeita.

A curtose de 1,12 é menor que 3, o que sugere que a distribuição dos retornos tem caudas mais leves do que uma distribuição normal (com curtose = 3). Em outras palavras, essa série tem menos extremos ou valores outliers comparada a uma distribuição normal. Uma curtose de 1,12 indica uma distribuição menos concentrada nas extremidades, com uma forma mais "achatada" no centro, em comparação com a distribuição normal.

Conclusões:
Distribuição levemente assimétrica: A assimetria positiva indica que a distribuição tem uma leve inclinação para a direita, sugerindo que, ocasionalmente, podem ocorrer retornos mais altos, mas não de forma predominante ou extrema.

Distribuição com caudas leves: A curtose baixa indica que as caudas da distribuição são mais leves do que uma distribuição normal. Isso pode ser bom para estratégias de risco, pois a probabilidade de grandes perdas (ou ganhos) é menor. Contudo, também significa que a distribuição tende a ser mais concentrada perto da média.

Em resumo, a série de retornos tem um comportamento levemente assimétrico para a direita e menos extremos em comparação com uma distribuição normal, o que pode indicar uma estratégia de risco mais controlado, mas também menos propensa a grandes oscilações.

**By: ChatGPT**

##### **Gráficos relacionados ao modelo MSGARCH e a LSTM**

*Vamos utilizar as ações da Coca-Cola Company e do JPMorgan para exemplificar, em um gráfico, como nosso modelo captura a volatilidade condicional. Em seguida, usaremos as ações do JPMorgan para ilustrar, também com um gráfico, como o modelo realiza previsões.*

In [ ]:
setup = {
    # Data inicial para a extração de dados (Deve obrigatoriamente ser um dia de negociação).
    "data_extraction_initial_date": datetime(2019,1,2).date(), #  Primeiro dia de negociações do S&P500 em 2019.
    
    # Data final cujos dados serão coletados (Deve obrigatoriamente ser um dia de negociação e 
    # deve também obrigatoriamente suceder um dia de negociação.).
    "data_extraction_final_date": datetime(2023,12,30).date(), #(Último dia de negociações do S&P500 em 2023). 

    # Define os períodos de tempo (janelas móveis) para o cálculo das principais features técnicas a serem usadas no modelo.
    "features_time_period": {
        # Define o período (janela móvel) para o cálculo dos retornos.
        "returns_time_period": 1,
        # Define o período (janela móvel) para o cálculo da média móvel exponencial.
        "exponential_moving_average_time_period": 14,
        # Define o período (janela móvel) para o cálculo do Índice de Força Relativa (RSI).
        "relative_strength_index_time_period": 14,
        # Define o período (janela móvel) para o cálculo da Faixa Média Verdadeira (ATR).
        "average_true_range_time_period": 14,
        # Define o período (janela móvel) para o cálculo do momentum.
        "momemtum_time_period": 14,
        # Frequência para coleta dos dados do VIX. O valor "1d" indica que os dados do VIX serão registrados 
        # em intervalos diários de negociação.
        "vix_time_period": "1d",
        # Define o período para o cálculo das mudanças percentuais (Usado no vix e no MSGARCH).
        "pct_change_period": 1,
    },
    
    # Período da estratégia a ser usada: define a frequência para a análise dos retornos (Ex.: 5 para análise semanal, 20 para mensal).
    "strategy_time_period": 20, # Observação: São levados em conta como dias na estratégia apenas aqueles onde ocorreram negociações.
    
    # Define o comprimento das sequências de tempo (janelas móveis) usadas no modelo LSTM.
    "lstm_time_sequences_length": 3
}

In [ ]:
# Cria um objeto ticker para as ações da Cola Cola Company.
coca_cola_data = Ticker("KO", setup['data_extraction_initial_date'], setup['data_extraction_final_date'], setup['features_time_period'],setup['strategy_time_period'])

# Cria um objeto ticker para as ações do JPMorgan.
jpmorgan_data = Ticker("JPM", setup['data_extraction_initial_date'], setup['data_extraction_final_date'], setup['features_time_period'],setup['strategy_time_period'])

In [ ]:
# Obtem o resultado da modelagem da volatilidade condicional do ticker "KO".
coca_cola_conditional_volatility = coca_cola_data.__get_MSGARCH_results__()[1]
coca_cola_conditional_volatility.name = "Coca Cola" # Seta o nome da série temporal "coca_cola_conditional_volatility".

# Obtem o resultado da modelagem da volatilidade condicional do ticker "JPM".
jpmorgan_conditional_volatility = jpmorgan_data.__get_MSGARCH_results__()[1]
jpmorgan_conditional_volatility.name = "JPMorgan" # Seta o nome da série temporal "jpmorgan_conditional_volatility".

In [ ]:
plot_multiples_time_series_line_graphs([coca_cola_conditional_volatility, jpmorgan_conditional_volatility], "Volatilidade Condicional das Ações da Coca-Cola e JPMorgan", "Data", "Volatilidade condicional")

In [ ]:
# Cria uma instância do modelo LSTM, passando o número de features e o comprimento das sequências de tempo (janelas temporais) para 
# preparar o modelo.
model = Model(9,setup['lstm_time_sequences_length'])

# Constrói a arquitetura da rede LSTM, configurando as camadas e os parâmetros necessários para o treinamento do modelo.
model.create_LSTM_model()

In [ ]:
def get_ticker_period_results(ticker: Ticker, predicted: pd.Series) -> list: 
    '''
        Description:
            ...
        Args:
            ticker (Ticker): Objeto do tipo Ticker que contém dados e informações relevantes do ativo.
            predicted (pd.Series): Série temporal contendo os valores preditos do preço do ativo.
        Return:
            pd.DataFrame: ... 
    '''

    # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
    resized_y_train = ticker.__y_train__.values.reshape(-1, 1)  

    # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
    scaler_target = MinMaxScaler()

    # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
    scaler_target.fit_transform(resized_y_train)

    # Volta os valores de "y_test_scaled_sequences" para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    real_y = pd.Series(scaler_target.inverse_transform(ticker.__y_test_scaled_sequences__).flatten(),
                    index= ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )
    
    # Adiciona um nome a série temporal "real_y".
    real_y.name = f"Preço real de {ticker.symbol}" # Útil para eventuais gráficos.

    # Volta os valores estimados para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    predicted_y = pd.Series(
        scaler_target.inverse_transform(predicted).flatten(),
        index=ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )
    
    # Adiciona um nome a série temporal "real_y".
    predicted_y.name = f"Preço predito de {ticker.symbol}" # Útil para eventuais gráficos.

    return real_y, predicted_y

In [ ]:
# Define o período a ser predito como sendo o período 61 da estratégia mensal.
test_initial_day = datetime(2023,11,28).date()
test_final_day = datetime(2023,12,27).date()


# Obtem os dados do ticker em questão em formato adequado para que o modelo LSTM possa realizer a predição.
X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences = jpmorgan_data.prepare_data_for_lstm(
    test_initial_day, test_final_day, setup['lstm_time_sequences_length'])
#
predicted, RMSE = model.train_model_and_get_results(X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences)

#
predicted_time_series, real_time_series = get_ticker_period_results(jpmorgan_data, predicted)

#
predicted_time_series.name = f"Valores preditos para {jpmorgan_data.symbol}"
real_time_series.name = f"Valores reais de {jpmorgan_data.symbol}"

results = {
    "Predicted": predicted_time_series,
    "Real": real_time_series,
    "RMSE": RMSE
}
    

In [ ]:
plot_multiples_time_series_line_graphs([results['Predicted'], results['Real']], "Valor Previsto vs. Valor Real das Ações do JPMorgan", "Data", "Preço de Fechamento Ajustado", f"RMSE: {results['RMSE']:.4f}")

#### *Código para gráficos de risco*

In [ ]:
# Funções para cálculo das métricas
def calculate_sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    return excess_returns.mean() / excess_returns.std() if excess_returns.std() != 0 else np.nan

def calculate_var(returns, confidence_level=0.95):
    return np.percentile(returns, (1 - confidence_level) * 100)

def calculate_cvar(returns, confidence_level=0.95):
    var = calculate_var(returns, confidence_level)
    return returns[returns <= var].mean()

# Parâmetros
risk_free_rate = 0
window_size = 6

# Calculando as métricas ao longo do tempo usando uma janela móvel
sharpe_ratios = average_monthly_results.rolling(window=window_size).apply(
    lambda x: calculate_sharpe_ratio(x, risk_free_rate), raw=False
)
vars_95 = average_monthly_results.rolling(window=window_size).apply(
    lambda x: calculate_var(x, confidence_level=0.95), raw=False
)
cvars_95 = average_monthly_results.rolling(window=window_size).apply(
    lambda x: calculate_cvar(x, confidence_level=0.95), raw=False
)

# Calculando o Sharpe ratio para o S&P 500
sp500_sharpe_ratios = sp500_index_period_returns.rolling(window=window_size).apply(
    lambda x: calculate_sharpe_ratio(x, risk_free_rate), raw=False
)

# Calculando as médias
mean_sharpe_ratio = sharpe_ratios.mean()
mean_sp500_sharpe_ratio = sp500_sharpe_ratios.mean()
mean_var = vars_95.mean()
mean_cvar = cvars_95.mean()

# Gráfico 1: Sharpe Ratio
fig_sharpe = go.Figure()

fig_sharpe.add_trace(go.Scatter(
    x=average_monthly_results.index,
    y=sharpe_ratios,
    mode='lines',
    name='Sharpe Ratio da Estratégia',
    line=dict(color='blue', width=2)
))

fig_sharpe.add_trace(go.Scatter(
    x=average_monthly_results.index,
    y=sp500_sharpe_ratios,
    mode='lines',
    name='Sharpe Ratio do S&P 500',
    line=dict(color='green', width=2)
))

fig_sharpe.add_trace(go.Scatter(
    x=[average_monthly_results.index[0], average_monthly_results.index[-1]],
    y=[mean_sharpe_ratio, mean_sharpe_ratio],
    mode='lines',
    name=f'Média Sharpe Ratio (Estratégia): {mean_sharpe_ratio:.2f}',
    line=dict(color='blue', width=1, dash='dash')
))

fig_sharpe.add_trace(go.Scatter(
    x=[average_monthly_results.index[0], average_monthly_results.index[-1]],
    y=[mean_sp500_sharpe_ratio, mean_sp500_sharpe_ratio],
    mode='lines',
    name=f'Média Sharpe Ratio (S&P 500): {mean_sp500_sharpe_ratio:.2f}',
    line=dict(color='green', width=1, dash='dash')
))

fig_sharpe.update_layout(
    title="Sharpe Ratio da Estratégia Vs. Sharpe Ratio do S&P 500 ao longo do tempo",
    xaxis_title="Data",
    yaxis_title="Sharpe Ratio",
    template="plotly_white",
    legend=dict(x=0.02, y=0.98)
)

fig_sharpe.show()

# Gráfico 2: VaR e CVaR
fig_var_cvar = go.Figure()

fig_var_cvar.add_trace(go.Scatter(
    x=average_monthly_results.index,
    y=vars_95,
    mode='lines',
    name='VaR (95%)',
    line=dict(color='red', dash='dash', width=2)
))

fig_var_cvar.add_trace(go.Scatter(
    x=average_monthly_results.index,
    y=cvars_95,
    mode='lines',
    name='CVaR (95%)',
    line=dict(color='purple', dash='dash', width=2)
))

fig_var_cvar.add_trace(go.Scatter(
    x=[average_monthly_results.index[0], average_monthly_results.index[-1]],
    y=[mean_var, mean_var],
    mode='lines',
    name=f'Média VaR (95%): {mean_var:.2f}',
    line=dict(color='red', width=1)
))

fig_var_cvar.add_trace(go.Scatter(
    x=[average_monthly_results.index[0], average_monthly_results.index[-1]],
    y=[mean_cvar, mean_cvar],
    mode='lines',
    name=f'Média CVaR (95%): {mean_cvar:.2f}',
    line=dict(color='purple', width=1)
))

fig_var_cvar.update_layout(
    title="VaR e CVaR ao longo do tempo",
    xaxis_title="Data",
    yaxis_title="Valor em Risco",
    template="plotly_white",
    legend=dict(x=0.02, y=0.98)
)

fig_var_cvar.show()

#### *Código para criação de tabela*

In [ ]:
# Carrega todos os arquivos e os combina em um único DataFrame usando o setup
dataframes = [pd.read_excel(file) for file in setup['monthly_results_sheets']]
combined_df = pd.concat(dataframes, ignore_index=True)

# Calcula a margem de erro como a diferença absoluta entre o retorno previsto e o retorno real
combined_df["Error Margin (%)"] = abs(combined_df["Retorno Predito"] - combined_df["Retorno Real"])

# Define os intervalos de erro e os rótulos para categorização
bins = [0, 1, 2, 5, 10, 20, 50, 100]
labels = ["0 - 1%", "1 - 2%", "2 - 5%", "5 - 10%", "10 - 20%", "20 - 50%", "50-100%"]
combined_df["Error Interval"] = pd.cut(combined_df["Error Margin (%)"], bins=bins, labels=labels, right=False)

# Conta a quantidade de ocorrências em cada intervalo de erro
error_margin_counts = combined_df["Error Interval"].value_counts().sort_index()

# Cria uma tabela de resumo
error_margin_table = pd.DataFrame({
    "Intervalo da Margem de Erro da Previsão": error_margin_counts.index,
    "Quantidade de Resultados": error_margin_counts.values
})

# Exibe a tabela de erro
error_margin_table

In [ ]:
pip install matplotlib pillow


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def dataframe_to_image(df, filename='tabela.png'):
    # Configura o tamanho da figura e remove os eixos
    fig, ax = plt.subplots(figsize=(len(df.columns), len(df) / 2))
    ax.axis('off')

    # Renderiza o DataFrame como uma tabela
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center'
    )

    # Ajusta a formatação da tabela
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(df.columns))))

    # Salva a tabela como uma imagem temporária
    plt.savefig("temp_table.png", bbox_inches='tight', dpi=300)
    plt.close(fig)

    # Abre a imagem temporária e converte para o formato PNG
    img = Image.open("temp_table.png")
    img.save(filename)

# Chama a função para criar e salvar a tabela como PNG
dataframe_to_image(error_margin_table)